# 30 Day Map Challenge: Day 1 (Points)

### Creating points using [What3Words' API](https://developer.what3words.com/public-api)
Reina Chano Murray  |  November 1, 2022  |  [github/reinacmurray](https://github.com/reinacmurray)  
 

### My approach to the 30 Day Map Challenge
In my day job, I spend more time advising people on making map applications and/or admninistering GIS systems, so I decided to use the 2022 [30 Day Map Challenge](https://30daymapchallenge.com/) as an avenue to 1) play around with creating maps programmatically and 2) explore software and platforms I haven't had a chance to use before. And of course, just have fun making maps! 

### The background on this script
Day 1 of the 30 Day Map Challenge is to create a map with points. I've been intrigued by the premise of [What3Words](https://what3words.com/) and decided to use today's challenge as an excuse to create an account and tap into their API. I've always been curious about the distribution of word combinations in What3Words (they have a nice [article here](https://what3words.medium.com/how-does-what3words-handle-similar-combinations-of-words-d480c94483c7) that delves into this a bit), and decided I wanted to plot all locations that share a common word. 

November 1 also happens to be my partner's birthday, so in their honor, I chose to plot their favorite word: penguin.  

I was envisioning a map where all what3words squares that have 'penguin' as one of their words (for example, 'penguin.secondword.thirdword', 'firstword.penguin.thirdword', or 'firstword.secondword.penguin') were plotted as points -- unfortunately, what I was envisioning wasn't quite possible with What3Words' API (and with a free-tier account). What3Words' [autosuggest](https://developer.what3words.com/public-api/docs#autosuggest) query had the functionality that most matched what I wanted to do -- it takes a partial 3 word address and returns up to 3 valid 3 word combinations. However, the first two words and at least the first character of the third word must be provided -- which meant I wouldn't be able to search for and pull all valid three word combinations that start with the word 'penguin', or have 'penguin' as their second word, and so on.

Cool. Fine. 

I improvised and decided to just settle for pulling and plotting a subset of valid three word combinations that contain the word 'penguin'. 

### What this script does
1. generate semi-random three word combinations that include 'penguin' as either the first or second word,
2. runs this word combination through What3Words' [autosuggest](https://developer.what3words.com/public-api/docs#autosuggest) function to return up to 3 valid three word combinations, 
3. runs these valid three word combinations through What3Words' [convert to coordinates](https://developer.what3words.com/public-api/docs#convert-to-coords) function to return their coordinates,
4. converts the result into a dataframe and exports it as a csv to be displayed in a GIS platform of your choice.

### What you need
If for whatever reason you want to use my method to create a map of word combinations from What3Words, you'll need to get an [API key from What3Words](https://developer.what3words.com/public-api).

You'll also need the python packages `random_word`, `what3words`, and `pandas`.

In [ ]:
# Set Up Required

w3w_api_key = ''  # put your API key here
selected_word = 'penguin' # enter your chosen word here

In [1]:
# import libraries
from enum import auto
import what3words
from random_word import RandomWords
import pandas as pd
from os import environ

In [2]:
# initiate random words object
r = RandomWords()

# initiate what3words geocoder object with your api key
geocoder = what3words.Geocoder(w3w_api_key)

In [3]:
# create list of letters in alphabet, lower case
alphabet_lc = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]

In [ ]:
# set up results list
no_results = []
results = []

In [1]:
# where your word is the first word in the three word combination
for each_letter in alphabet_lc:
    three_words = selected_word + "." + r.get_random_word() + "." + each_letter
    
    autosuggest = geocoder.autosuggest(three_words)

    if 'error' in autosuggest: # An error has been returned from the API
        code = autosuggest['error']['code']
        message = autosuggest['error']['message']
        no_results.append([code, message, three_words])
    
    else:
        results.append(autosuggest)

In [12]:
# where your word is the second word in the three word combination
for each_letter in alphabet_lc:
    three_words = r.get_random_word() + "." + selected_word + "." + each_letter
    
    autosuggest = geocoder.autosuggest(three_words)

    if 'error' in autosuggest: # An error has been returned from the API
        code = autosuggest['error']['code']
        message = autosuggest['error']['message']
        no_results.append([code, message, three_words])
    
    else:
        results.append(autosuggest)

In [72]:
results

[{'suggestions': [{'country': 'RU',
    'nearestPlace': 'Katyryk, Krasnoyarsk Krai',
    'words': 'penguin.belches.also',
    'rank': 1,
    'language': 'en'},
   {'country': 'RU',
    'nearestPlace': 'Lorino, Chukotka Autonomous Okrug',
    'words': 'penguin.belches.back',
    'rank': 2,
    'language': 'en'},
   {'country': 'FI',
    'nearestPlace': 'Pudasjärvi, Northern Ostrobothnia',
    'words': 'penguin.belches.each',
    'rank': 3,
    'language': 'en'}]},
 {'suggestions': [{'country': 'US',
    'nearestPlace': 'Bradford, Tennessee',
    'words': 'penguin.assembles.back',
    'rank': 1,
    'language': 'en'},
   {'country': 'GL',
    'nearestPlace': 'Northeast Greenland National Park',
    'words': 'penguin.assembled.back',
    'rank': 2,
    'language': 'en'},
   {'country': 'US',
    'nearestPlace': 'Hebbronville, Texas',
    'words': 'penguin.assembles.both',
    'rank': 3,
    'language': 'en'}]},
 {'suggestions': [{'country': 'CA',
    'nearestPlace': 'Pangnirtung, Nunavut'

In [31]:
# create a list of words from result to geocode
to_geocode = []

for a in range(len(results)):
    for b in range(len(results[a]['suggestions'])):
        
        try:
            to_geocode.append(results[a]["suggestions"][b]["words"])
        except:
            pass

['penguin.belches.also', 'penguin.belches.back', 'penguin.belches.each', 'penguin.assembles.back', 'penguin.assembled.back', 'penguin.assembles.both', 'penguin.sanest.back', 'penguin.starkest.back', 'penguin.sanest.much', 'penguin.dreamtime.deep', 'penguin.dreamtime.good', 'penguin.dreamtime.down', 'penguin.investor.deep', 'penguin.investor.even', 'penguin.investor.each', 'penguin.able.from', 'penguin.deal.from', 'penguin.able.life', 'penguin.chronometer.good', 'penguin.chronometer.give', 'penguin.chronometer.long', 'penguin.spearing.than', 'penguin.spearing.much', 'penguin.spearing.such', 'penguin.industrious.into', 'penguin.industrious.like', 'penguin.industrious.life', 'penguin.bestows.just', 'penguin.bestow.just', 'penguin.bestows.jobs', 'penguin.metropolis.like', 'penguin.metropolis.take', 'penguin.paradoxical.also', 'penguin.paradoxical.last', 'penguin.paradoxical.help', 'penguin.hypocrite.from', 'penguin.hypocrite.many', 'penguin.hypocrite.much', 'penguin.reduce.into', 'penguin.

In [65]:
# geocode each 3words in the list
geocoded_results = []

for each_3word in to_geocode:
    temp_dict = {}
    try:
        geocoded = geocoder.convert_to_coordinates(each_3word)
        temp_dict["words"] = each_3word
        temp_dict["southwest_lng"] = geocoded["square"]["southwest"]["lng"]
        temp_dict["southwest_lat"] = geocoded["square"]["southwest"]["lat"]
        temp_dict["northeast_lng"] = geocoded["square"]["northeast"]["lng"]
        temp_dict["northeast_lat"] = geocoded["square"]["northeast"]["lat"]
        temp_dict["nearestPlace"] = geocoded["nearestPlace"]
        temp_dict["pt_lng"] = geocoded["coordinates"]["lng"]
        temp_dict["pt_lat"] = geocoded["coordinates"]["lat"]
        temp_dict["pt_lat"] = geocoded["coordinates"]["lat"]
        temp_dict["country"] = geocoded["country"]
        temp_dict["language"] = geocoded["language"]
        geocoded_results.append(temp_dict)
    except:
        pass

In [66]:
# view results
geocoded_results

[{'words': 'penguin.belches.also',
  'southwest_lng': 99.855669,
  'southwest_lat': 69.466661,
  'northeast_lng': 99.855746,
  'northeast_lat': 69.466688,
  'nearestPlace': 'Katyryk, Krasnoyarsk Krai',
  'pt_lng': 99.855707,
  'pt_lat': 69.466675,
  'country': 'RU',
  'language': 'en'},
 {'words': 'penguin.belches.back',
  'southwest_lng': -173.058264,
  'southwest_lat': 66.955476,
  'northeast_lng': -173.058196,
  'northeast_lat': 66.955503,
  'nearestPlace': 'Lorino, Chukotka Autonomous Okrug',
  'pt_lng': -173.05823,
  'pt_lat': 66.95549,
  'country': 'RU',
  'language': 'en'},
 {'words': 'penguin.belches.each',
  'southwest_lng': 27.564406,
  'southwest_lat': 65.305708,
  'northeast_lng': 27.56447,
  'northeast_lat': 65.305735,
  'nearestPlace': 'Pudasjärvi, Northern Ostrobothnia',
  'pt_lng': 27.564438,
  'pt_lat': 65.305722,
  'country': 'FI',
  'language': 'en'},
 {'words': 'penguin.assembles.back',
  'southwest_lng': -88.889011,
  'southwest_lat': 36.085382,
  'northeast_lng': 

In [73]:
df = pd.DataFrame(geocoded_results)

In [74]:
# forgot to specify in autosuggest to only keep results where the language is english. 
not_penguins = df[df.language != "en"]
not_penguins

,words,southwest_lng,southwest_lat,northeast_lng,northeast_lat,nearestPlace,pt_lng,pt_lat,country,language
110,supplétive.engin.lier,-69.820167,-49.706716,-69.820125,-49.706689,"Comandante Luis Piedra Buena, Santa Cruz",-69.820146,-49.706703,AR,fr
111,supplétive.pingouin.lier,-103.617043,23.365594,-103.617013,23.365621,"Jiménez del Teul, État de Zacatecas",-103.617028,23.365607,MX,fr
145,baby.pinguin.aue,39.039659,22.050857,39.039688,22.050884,"Khulays, Mecca",39.039674,22.050871,SA,de
146,baby.pinguin.exil,24.253066,63.350232,24.253127,63.350259,"Halsua, Mittelösterbotten",24.253096,63.350245,FI,de
147,baby.pinguinen.aue,-62.849026,-5.014796,-62.848999,-5.014769,"Coari, Amazonas",-62.849012,-5.014783,BR,de


In [69]:
# remove any where the language is not English
df = df[df.language == "en"]

In [70]:
# examine dataframe
df

,words,southwest_lng,southwest_lat,northeast_lng,northeast_lat,nearestPlace,pt_lng,pt_lat,country,language
0,penguin.belches.also,99.855669,69.466661,99.855746,69.466688,"Katyryk, Krasnoyarsk Krai",99.855707,69.466675,RU,en
1,penguin.belches.back,-173.058264,66.955476,-173.058196,66.955503,"Lorino, Chukotka Autonomous Okrug",-173.058230,66.955490,RU,en
2,penguin.belches.each,27.564406,65.305708,27.564470,65.305735,"Pudasjärvi, Northern Ostrobothnia",27.564438,65.305722,FI,en
3,penguin.assembles.back,-88.889011,36.085382,-88.888978,36.085409,"Bradford, Tennessee",-88.888995,36.085395,US,en
4,penguin.assembled.back,-16.191074,81.092524,-16.190900,81.092551,Northeast Greenland National Park,-16.190987,81.092537,GL,en
...,...,...,...,...,...,...,...,...,...,...
149,gain.penguin.many,116.828700,-24.771615,116.828730,-24.771588,"Paraburdoo, Western Australia",116.828715,-24.771601,AU,en
150,gain.penguin.only,123.200490,-22.403649,123.200519,-22.403622,"Kumpupintil, Western Australia",123.200504,-22.403636,AU,en
151,satellite.penguin.zone,131.558130,-28.873410,131.558161,-28.873383,"Watarru, South Australia",131.558146,-28.873396,AU,en
152,satellite.penguin.jazz,115.382276,-22.629851,115.382305,-22.629824,"Paraburdoo, Western Australia",115.382290,-22.629838,AU,en


In [71]:
# save to csv
df.to_csv("penguins_geocoded.csv", encoding = "utf-8", index = False)